In [ ]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 15.3 MB/s eta 0:00:00


In [ ]:
pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 34.3 MB/s eta 0:00:00


In [ ]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.1 MB/s eta 0:00:00


In [ ]:
pip install pypdf2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from PyPDF2 import PdfReader


pdf_reader = PdfReader("/content/UserManual.pdf")

In [ ]:
text = ''
for i, page in enumerate(pdf_reader.pages):
  text = page.extract_text()
  if text:
    text += text



In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
chunk_size = 512,
chunk_overlap  = 32,
length_function = len,
)
texts = text_splitter.split_text(text)

In [ ]:
embeddings = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
docsearch = FAISS.from_texts(texts, embeddings)

In [ ]:
from langchain import HuggingFaceHub
import os

In [53]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'your API key'
llm=HuggingFaceHub(repo_id="mistralai/Mistral-7B-v0.1",
                                        model_kwargs={"temperature":1,
                                                      "top_p":0.9,
                                                      "max_new_tokens": 250,
                                                      "repetition_penalty":1.1,"return_full_text":False})

In [54]:
chain = load_qa_chain(llm, chain_type="stuff")


In [45]:
query  = "What is the manual about?"
docs  = docsearch.similarity_search(query)
print(chain.run(input_documents=docs, question=query))

 Oracle VM VirtualBox User Manual
Oracle VM VirtualBox is a hosted hypervisor (type 2) which allows multiple virtual ma-
chines with their own operating systems to run simultaneously on one physical compu-
ter (the host).
An application running on the host would not be aware that such virtualization is taking
place since it only interacts with the resources of its “real” machine. From the point of view
of the guest operating system in a VM, it operates as if it were on a regular hardware plat-
form. All instructions sent to the VMs appear to come from real hardware devices and the
output of the guests is rendered on the real host system’s display.
These two facts make VirtualBox highly suitable for testing and training purposes as the
visitor machines on your system will seem and behave exactly like separate systems. In addi-
tion, VirtualBox offers the advantages of being extremely lightweight in terms of system
resources consumed, yet completely portable on different types of host op

In [55]:
query  = "How to install virtual box"
docs  = docsearch.similarity_search(query)
print(chain.run(input_documents=docs, question=query))

 I prefer to use the 'File=> Install Oracle VM VirtualBox Extensions" method. Here's how that goes: Start a virtual machine. Navigate to Oracle VM VirtualBox Menu => Devices=> Insert Guest Additions CD image.
